In [3]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from kneed import KneeLocator
from sklearn.neighbors import NearestNeighbors
from numpy import inf

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

import pandas_datareader.data as web
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from pandas import ExcelFile
import os
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from matplotlib.pyplot import figure

import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score, silhouette_samples
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler,MaxAbsScaler
from sklearn.metrics import silhouette_score
import random


from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from kneed import KneeLocator

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from sklearn.datasets import make_blobs
from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import itertools

COLOR_BOLD = "\033[95m"
COLOR_END = "\033[0m"

In [4]:
pd.options.display.float_format = '{:,.10f}'.format

In [5]:
pd.set_option('display.max_colwidth', -1)

# Functions

In [6]:
def loop_files(string_to_find):
    '''
    Returns list of file paths containing certain string
    '''
    filepaths = []
    for root, dirs, files in os.walk('Export/'):
        for name in files:
            if name.endswith('.csv'):

                filepath = os.path.join(root, name)
                if f'{string_to_find}' in name:
                    filepaths.append(filepath)
    return filepaths

In [7]:
def read_returns(file_path):
    """
    Returns a dataframe when given a file path to csv file of returns
    """
    movements = pd.read_csv(f'{file_path}',
                              header=0,
                              index_col=0,
                              parse_dates=True)
    
    return movements

In [8]:
def find_knee(movements, min_samples, direction = 'increasing'):
    '''
    Finds knee for data set. AKA the epsilon, or distance
    points need to be from centroid to be considered a cluster
    '''
    
    nearest_neighbors = NearestNeighbors(n_neighbors=min_samples)
    neighbors = nearest_neighbors.fit(movements)
    distances, indices = neighbors.kneighbors(movements)

    distances = np.sort(distances[:,1], axis=0)



    i = np.arange(len(distances))
    knee = KneeLocator(i, distances,
                       S=1, curve='convex',
                       direction=direction,
                       interp_method='polynomial')
    '''
    fig = plt.figure(figsize=(5, 3))

    knee.plot_knee()

    plt.xlabel("Points")
    plt.ylabel("Distance")
    '''
    return distances[knee.knee]

In [9]:
def tune_n_clusters(movements, upper_bound = 21, random_state = 10, init = 'k-means++'):
    
    '''
    returns optimal cluster number
    '''
    
    n_clusters = list(range(2,upper_bound))

    sil_dict = {}

    for cluster in n_clusters:
        clusterer = KMeans(n_clusters = cluster, random_state=random_state, init=init)
        cluster_labels = clusterer.fit(movements)

        sil_avg = silhouette_score(movements, clusterer.labels_)

        sil_dict[cluster] = sil_avg
        
    #return max(sil_dict, key = sil_dict.get)
    return sil_dict

In [10]:
def get_n_clusters(movements, random_state=10, max_iter=1000):
    '''
    Solves for n_clusters progrimatically using
    SSE elbow method, and Shilouette method.
    
    Uses both random init and k-means++ init.
    '''
    n_clusters = []
    #------
    kmeans_kwargs = {
        "init": 'k-means++',
        "max_iter": max_iter,
        "random_state": random_state,
    }

    # A list holds the SSE values for each k
    sse = []
    for k in range(1, 15):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(movements)
        sse.append(kmeans.inertia_)

    kl = KneeLocator(range(1, 15), sse, curve="convex", direction="decreasing")

    n_clusters.append(kl.elbow)
    #------
    kmeans_kwargs = {
        "init": "random",
        "max_iter": 1000,
        "random_state": random_state,
    }

    # A list holds the SSE values for each k
    sse = []
    for k in range(1, 15):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(movements)
        sse.append(kmeans.inertia_)

    kl = KneeLocator(range(1, 15), sse, curve="convex", direction="decreasing")

    n_clusters.append(kl.elbow)
    #------
    kmeans_kwargs = {
        "init": "k-means++",
        "max_iter": 1000,
        "random_state": random_state,
    }

    # A list holds the silhouette coefficients for each k
    silhouette_coefficients = []

    # Notice you start at 2 clusters for silhouette coefficient
    for k in range(2, 15):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(movements)
        score = silhouette_score(movements, kmeans.labels_)
        silhouette_coefficients.append(score)
        
    #return len(silhouette_coefficients), len(list(range(2, 15)))
    zipped = pd.DataFrame({
        '0': list(range(2, 15)),
        '1': silhouette_coefficients
    })

    n_clusters.append(int(zipped.loc[zipped['1'].idxmax()][0]))

    kmeans_kwargs = {
        "init": "random",
        "max_iter": 1000,
        "random_state": 42,
    }

    # A list holds the silhouette coefficients for each k
    silhouette_coefficients = []

    # Notice you start at 2 clusters for silhouette coefficient
    for k in range(2, 15):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(movements)
        score = silhouette_score(movements, kmeans.labels_)
        silhouette_coefficients.append(score)

    zipped = pd.DataFrame({
        '0': list(range(2, 15)),
        '1': silhouette_coefficients
    })
    
    n_clusters.append(int(zipped.loc[zipped['1'].idxmax()][0]))
    #------

    return list(set(n_clusters))

In [11]:
def opti_cluster(movements, eps, upper_bound = 50, DBS = True):
    '''
    Returns number of cluster for certain number of min_samples
    '''
    labels_dict = {}
    ms = list(range(1,upper_bound))
    
    if DBS:
        for i in ms:
            db = DBSCAN(eps = eps, min_samples=i)
            labels = db.fit_predict(movements)


            labels = set([label for label in labels if label >= 0])
            labels_dict[i] = len(labels)
        return labels_dict
        
    if not DBS:
        for i in ms:
            db = OPTICS(eps = eps, min_samples=i)
            labels = db.fit_predict(movements)


            labels = set([label for label in labels if label >= 0])

            labels_dict[i] = len(labels)
        return labels_dict

In [12]:
def mod_z_score(sqdist, consistency_correction = 1.4826):
    median = np.median(sqdist)
    
    
    dev_from_med = np.array(sqdist) - median
    
    mad = np.median(np.abs(dev_from_med))
    
    mod_zscore = dev_from_med/(consistency_correction*mad)
    
    return mod_zscore#, mad

In [13]:
def KMEANS_DS(movements,
              n_clusters=3,
              init='k-means++',
              n_init=50,
              max_iter=1000,
              random_state=10,
              algorithm='lloyd',
              PCA_components = 3,
              scaler = MinMaxScaler(),
              ZSCORE = 3):
    '''
    Returns tuple of Silhoutte Score and Predicted Labels
    '''
    preprocessor = Pipeline([
        ("scaler", scaler),
        ("pca", PCA(n_components=PCA_components, random_state=random_state)),
    ])

    clusterer = Pipeline([
        (
            "kmeans",
            KMeans(
                n_clusters=n_clusters,
                init=init,
                n_init=n_init,
                max_iter=max_iter,
                random_state=random_state,
            ),
        ),
    ])

    pipe = Pipeline([("preprocessor", preprocessor), ("clusterer", clusterer)])
    
    
    
    fitted_movements = pipe.fit(movements)
    labels = pipe.predict(movements)
    preprocessed_data = pipe["preprocessor"].transform(movements)

    predicted_labels = pd.DataFrame(index=range(nrow),
                                    columns=['ticker', 'group'])
    

    
    i = 0
    for ticker in list(movements.index):
        current_prediction = fitted_movements.predict(
            [list(movements.iloc[i])])[0]
        predicted_labels.loc[i, 'ticker'] = ticker
        predicted_labels.loc[i, 'group'] = current_prediction
        i = i + 1
        
    X_dist = preprocessed_data**2    
    #centers = pipe.named_steps['clusterer'][0].cluster_centers_
    #outliers = KM_OUTLIERS(centers, predicted_labels['group'])
    
    predicted_labels['sqdist'] = X_dist.sum(axis=1)
    
    
    #modZ = predicted_labels.groupby('group')['sqdist']
    #zscore = lambda x: mod_z_score(x)
    predicted_labels['modZZ'] = predicted_labels.groupby('group')['sqdist'].transform(mod_z_score)
    predicted_labels = predicted_labels[predicted_labels['modZZ'] < ZSCORE]
    #predicted_labels = predicted_labels[predicted_labels.group > -1]
    #sil_score = silhouette_score(preprocessed_data, labels = predicted_labels['group'])

    return  predicted_labels #sil_score

In [14]:
def DBSCAN_DS(
        movements,
        eps=0.5,
        min_samples=5,
        metric='euclidean',
        metric_params=None,
        algorithm='auto',
        leaf_size=30,
        p=None,
        n_jobs=None,
        scaler=MinMaxScaler(),
        PCA_components=3,
        random_state=10):

    preprocessor = Pipeline([
        ("scaler", scaler),
        ("pca", PCA(n_components=PCA_components, random_state=random_state, svd_solver='auto')),
    ])

    clusterer = Pipeline([
        (
            "DBSCAN",
            DBSCAN(
                eps=eps,
                min_samples=min_samples,
                metric=metric,
                algorithm=algorithm,
                p=p,
                leaf_size=leaf_size,
            ),
        ),
    ])

    

    pipe = Pipeline([("preprocessor", preprocessor), ("clusterer", clusterer)])
    
    
    labels = pipe.fit_predict(movements)
    #preprocessed_data = pipe["preprocessor"].transform(movements)
    d = {'ticker': movements.index.values, 'group': labels}
    predicted_labels = pd.DataFrame(data=d,
                                    index=range(nrow),
                                    columns=['ticker', 'group'])
    
    predicted_labels = predicted_labels[predicted_labels.group>-1]
    #sil_score = silhouette_score(preprocessed_data, predicted_labels['group'])

    return predicted_labels

    

In [15]:
def OPTICS_DS(movements,
              scaler=MinMaxScaler(),
              PCA_components=3,
              random_state=10,
              min_samples=5,
              max_eps=inf,
              metric='minkowski',
              p=2,
              metric_params=None,
              cluster_method='xi',
              eps=None,
              xi=0.05,
              predecessor_correction=True,
              min_cluster_size=None,
              algorithm='auto',
              leaf_size=30,
              n_jobs = -1):

    preprocessor = Pipeline([
        ("scaler", scaler),
        ("pca", PCA(n_components=PCA_components, random_state=random_state)),
    ])

    clusterer = Pipeline([
        (
            "OPTICS",
            OPTICS(
                min_samples=min_samples,
                max_eps=max_eps,
                metric=metric,
                p=p,
                metric_params=metric_params,
                cluster_method=cluster_method,
                eps=eps,
                xi=xi,
                predecessor_correction=predecessor_correction,
                min_cluster_size=min_cluster_size,
                algorithm=algorithm,
                leaf_size=leaf_size,
                memory=None,
                n_jobs=n_jobs,
            ),
        ),
    ])

    pipe = Pipeline([("preprocessor", preprocessor), ("clusterer", clusterer)])
    
    
    labels = pipe.fit_predict(movements)

    #preprocessed_data = pipe["preprocessor"].transform(movements)
    d = {'ticker': movements.index.values, 'group': labels}
    predicted_labels = pd.DataFrame(data=d,
                                    index=range(nrow),
                                    columns=['ticker', 'group'])
    
    
    return predicted_labels

In [16]:
def SUMMARYSTATS_DS(predicted_labels, returns_file_name, sharpes_file_name):
    '''
    Returns Constituent Companies of Portfolio and their return
    in during covid in excess of SP500 avg return during same time frame
    '''
    #Portfolio Construction
    sharpes = pd.read_csv(f"{sharpes_file_name}",
                          header=0,
                          index_col=0,
                          parse_dates=True)

    merged = sharpes.merge(predicted_labels[['group','ticker']], on=['ticker'])

    max_sharpes = merged.groupby('group')['sharpes', 'ticker'].max()

    # Testing Portfolio Returns During Covid

    movements = pd.read_csv(f'Export/returns_covid_PC.csv',
                            header=0,
                            index_col=0,
                            parse_dates=True)

    portfolio = movements[movements.index.isin(max_sharpes['ticker'])]

    portfolio = portfolio.T

    overall = (portfolio.iloc[:, :] + 1).prod() - 1
    average = overall.mean()

    #Excess Return


    excess = average - spExcess


    return portfolio.columns, excess

# SP500 Returns

In [17]:
# import historical s&p 500 data as data frame
sp500_hist = pd.read_csv('sp500_historical.csv',
                          header=0,
                          index_col=0,
                          parse_dates=True)

sp500_hist = sp500_hist['2020-02-03':'2020-04-14']

spExcess = (sp500_hist['Adj Close'].pct_change()+1).prod()-1

# OHLC

## KMEANS

### KMEANS Params

In [18]:
###### KMEANS Params List
n_clusters = [2,11,16,21]
pca_components = [6,7,8,14]
scaler = [MinMaxScaler()]
init = ['k-means++']
ZS = [1,1.3,1.5,2,3.5]
ninit = [5,50]

In [19]:
ret_files = loop_files('ac_de')#['Export/ac_detrend_ret.csv']

In [20]:
sharpe_files= loop_files('sharpes_monthly_diff_OCO')#['Export/sharpes_monthly_diff_OCO.csv']

In [21]:
sharpe_files

['Export/sharpes_monthly_diff_OCO.csv']

In [22]:
#iter_list[378] #best kmeans combo for this iterlist
iter_KM = list(itertools.product(n_clusters, #0
                                   pca_components, #1
                                   scaler, #2
                                   init, #3
                                   ret_files, #4
                                   sharpe_files, #5
                                 ZS,#6
                                ninit)) #7

In [23]:
len(iter_KM)

160

### KMEANS Loop for all Combos of Params

In [193]:
sil_master_KM = []
predicted_labels_master_KM = []
portfolio_companies_master_KM = []
excess_master_KM = []


for combo in iter_KM:
    movements = read_returns(combo[4])
    movements = movements[:-1]
    nrow = movements.shape[0]
    ncol = movements.shape[1]

    predicted_labels = KMEANS_DS(movements,
                                            n_clusters=combo[0],
                                            init=combo[3],
                                            n_init= combo[7],
                                            scaler=combo[2],
                                            PCA_components=combo[1],
                                            ZSCORE = combo[6],
                                random_state=10)

    portfolio_companies, excess = SUMMARYSTATS_DS(predicted_labels, combo[4],
                                                  combo[5])

    #sil_master_KM.append(sil_score)
    predicted_labels_master_KM.append(predicted_labels)
    portfolio_companies_master_KM.append(portfolio_companies)
    excess_master_KM.append(excess)



C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invali

C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19988\2292402150.py:9: RuntimeWarning: invali

### KMEANS Results

In [194]:
max(excess_master_KM)

0.016249711211300144

In [195]:
max_index = excess_master_KM.index(max(excess_master_KM))

In [196]:
max_index

51

In [197]:
portfolio_companies_master_KM[max_index]

Index(['MAS', 'NVDA', 'TJX', 'TMO', 'VRSN', 'WFC', 'WMB', 'WMT', 'XEL', 'XRX',
       'YUM'],
      dtype='object')

In [198]:
iter_KM[max_index]

(11,
 7,
 MinMaxScaler(),
 'k-means++',
 'Export/ac_detrend.csv',
 'Export/sharpes_monthly_diff_OCO.csv',
 1,
 50)

## DBSCAN

### DBSCAN Params

In [ ]:
#DB param lists
scaler = [MinMaxScaler(), MaxAbsScaler()]
PCA_components = [4,5,6,11]
min_samples = [3,10,11,12,18]
#max_eps = [2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5]
metric = ['euclidean','minkowski']

eps = [0.43333, 0.4245796527260531,0.3357278927089582,0.2630817273081133,0.25818012165540655,0.20696961857497598,0.14048568590637187]
algorithm = ['auto']
n_jobs = [-1]
p = [1.5]

In [ ]:
ret_files = loop_files('ac_detrend')

In [ ]:
sharpe_files= loop_files('sharpes_diff_O')

In [ ]:
iter_DB = list(
    itertools.product(
        scaler, #0
        PCA_components, #1
        min_samples, #2
        metric, #3
        eps, #4
        algorithm, #5
        n_jobs, #6
        ret_files,  #7
        sharpe_files,#8
        p)) #9

In [ ]:
len(iter_DB)

### DBSCAN Loop for all Combos of Params

In [ ]:
sil_master_DB = []
predicted_labels_master_DB = []
portfolio_companies_master_DB = []
excess_master_DB = []
combo_excess_masterlist_DB = []

for combo in iter_DB:
    movements = read_returns(combo[7])
    movements = movements[:-1]
    nrow = movements.shape[0]
    ncol = movements.shape[1]

    predicted_labels = DBSCAN_DS(movements,
                                 eps=combo[4],
                                 min_samples=combo[2],
                                 metric=combo[3],
                                 algorithm=combo[5],
                                 n_jobs=combo[6],
                                 scaler=combo[0],
                                 PCA_components=combo[1],
                                 p=combo[9])
    #predicted_labels = predicted_labels[predicted_labels['group'] > -1]
    #  ^gets rid of outliers group
    
    
    portfolio_companies, excess = SUMMARYSTATS_DS(predicted_labels, combo[7],
                                                  combo[8])

    #sil_master_DB.append(sil_score)
    predicted_labels_master_DB.append(predicted_labels)
    portfolio_companies_master_DB.append(portfolio_companies)
    excess_master_DB.append(excess)
    combo_excess_masterlist_DB.append([combo, excess])

### DBSCAN Results

In [ ]:
max(excess_master_DB)

In [ ]:
max_index = excess_master_DB.index(max(excess_master_DB))

In [ ]:
max_index

In [ ]:
portfolio_companies_master_DB[max_index]

In [ ]:
iter_DB[max_index]

## OPTICS

### OPTICS Params

In [277]:
#OPTICS Params
scaler = [MinMaxScaler(),MaxAbsScaler()]
PCA_components = [17]
random_state= [10]
min_samples= [14]
max_eps = inf
metric= ['minkowski']
p=2,
metric_params= None 
cluster_method= 'xi' 
eps= [0.3357278927]#[0.0633380090583151,0.4245796527260531,0.3357278927089582,0.2630817273081133,0.25818012165540655,0.20696961857497598]
xi= [0.05]
predecessor_correction = [True]
min_cluster_size = [0.05]
algorithm = ['auto'] 
leaf_size= [30] 


In [278]:
ret_files = loop_files('ac_detrend')

In [279]:
sharpe_files= ['Export/sharpes_monthly_diff_OCO.csv']#loop_files('sharpe')

In [280]:
iter_OPT = list(
    itertools.product(
        scaler, #0
        PCA_components, #1
        min_samples, #2
        metric, #3
        eps, #4
        xi,#5
        min_cluster_size,#6
        algorithm, #&
        ret_files,  #8
        sharpe_files,#9
        p)) #10

In [281]:
len(iter_OPT)

2

### OPTICS Loop for all Combos of Params

In [282]:
sil_master_OPT = []
predicted_labels_master_OPT = []
portfolio_companies_master_OPT = []
excess_master_OPT = []
combo_excess_masterlist_OPT = []

for combo in iter_OPT:
    movements = read_returns(combo[8])

    nrow = movements.shape[0]
    ncol = movements.shape[1]

    predicted_labels = OPTICS_DS(movements,
                                            scaler=combo[0],
                                            PCA_components=combo[1],
                                            min_samples=combo[2],
                                            metric=combo[3],
                                            eps=combo[4],
                                            xi=combo[5],
                                            min_cluster_size=combo[6])
    
    predicted_labels = predicted_labels[predicted_labels['group'] > -1]
    #  ^gets rid of outliers group
    portfolio_companies, excess = SUMMARYSTATS_DS(predicted_labels, combo[8],
                                                  combo[9])

    #sil_master_DB.append(sil_score)
    predicted_labels_master_OPT.append(predicted_labels)
    portfolio_companies_master_OPT.append(portfolio_companies)
    excess_master_OPT.append(excess)
    combo_excess_masterlist_OPT.append([combo,excess])

### OPTICS Results

In [284]:
max(excess_master_OPT)

0.17171391070979286

In [285]:
max_index = excess_master_OPT.index(max(excess_master_OPT))

In [286]:
max_index

0

In [287]:
portfolio_companies_master_OPT[max_index]

Index(['WMT', 'XEL'], dtype='object')

In [288]:
iter_OPT[max_index]

(MinMaxScaler(),
 17,
 14,
 'minkowski',
 0.3357278927,
 0.05,
 0.05,
 'auto',
 'Export/ac_detrend.csv',
 'Export/sharpes_monthly_diff_OCO.csv',
 2)

# Ratios Test

## K-Means

### KMEANS Params

In [36]:
###### KMEANS Params List
n_clusters = [4,7]
pca_components = [2,3,4,11]
scaler = [MinMaxScaler()]
init = ['k-means++']
ZS = [3,4,5]
ninit = [50]

In [37]:
ratios_files = loop_files('ratios.csv')#['Export/ac_detrend_ret.csv']

In [38]:
sharpe_files= loop_files('sharpes_monthly_diff_OCO')

In [39]:
ratios_files

['Export/ratios.csv']

In [40]:
iter_KM = list(itertools.product(n_clusters, #0
                                   pca_components, #1
                                   scaler, #2
                                   init, #3
                                   ratios_files, #4
                                   sharpe_files, #5
                                 ZS,#6
                                ninit)) #7

In [41]:
len(iter_KM)

24

### KMEANS Loop for all Combos of Params

In [42]:
sil_master_KM = []
predicted_labels_master_KM = []
portfolio_companies_master_KM = []
excess_master_KM = []


for combo in iter_KM:
    movements = read_returns(combo[4])
    movements = movements[:-1]
    nrow = movements.shape[0]
    ncol = movements.shape[1]

    predicted_labels = KMEANS_DS(movements,
                                            n_clusters=combo[0],
                                            init=combo[3],
                                            n_init= combo[7],
                                            scaler=combo[2],
                                            PCA_components=combo[1],
                                            ZSCORE = combo[6],
                                random_state=10)

    portfolio_companies, excess = SUMMARYSTATS_DS(predicted_labels, combo[4],
                                                  combo[5])

    #sil_master_KM.append(sil_score)
    predicted_labels_master_KM.append(predicted_labels)
    portfolio_companies_master_KM.append(portfolio_companies)
    excess_master_KM.append(excess)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16808\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16808\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16808\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16808\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16808\2292402150.py:9: RuntimeWarning: invalid value encountered in true_divide
  mod_zscore = dev_from_med/(consistency_correction*mad)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16808\2292402150.py:9: RuntimeWarning: invali

### KMEANS Results

In [43]:
max(excess_master_KM)

0.03450409782912439

In [44]:
max_index = excess_master_KM.index(max(excess_master_KM))

In [45]:
max_index

2

In [46]:
portfolio_companies_master_KM[max_index]

Index(['TJX', 'WMT', 'XEL', 'XOM'], dtype='object')

In [47]:
iter_KM[max_index]

(4,
 2,
 MinMaxScaler(),
 'k-means++',
 'Export/ratios.csv',
 'Export/sharpes_monthly_diff_OCO.csv',
 5,
 50)

## DBSCAN

### DBSCAN Params

In [328]:
#DB param lists
scaler = [MinMaxScaler(), MaxAbsScaler()]
PCA_components = [2,4,6,7,8,9]
min_samples = [10,11,12]
max_eps = [2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5]
metric = [
    'minkowski','euclidean','manhattan'
]
#eps = [0.04461719757124152]
eps = [0.42383600018117623, 0.33948192944865985,0.43280100014311673,0.256098350775273,0.5621840302358065,0.3998988812520709]
algorithm = ['auto']
n_jobs = [-1]
p = [1.5]


In [ ]:
movements = read_returns('Export/ac_detrend.csv')

In [ ]:
nearest_neighbors = NearestNeighbors(n_neighbors=5)
neighbors = nearest_neighbors.fit(movements)
distances, indices = neighbors.kneighbors(movements)

distances = np.sort(distances[:,1], axis=0)



i = np.arange(len(distances))
knee = KneeLocator(i, distances,
                   S=3, curve='convex',
                   direction='increasing',
                   interp_method='interp1d')
#'''
fig = plt.figure(figsize=(5, 3))

knee.plot_knee()

plt.xlabel("Points")
plt.ylabel("Distance")
#'''

distances[knee.knee]

In [338]:
ratios_files = loop_files('ratios.csv')#['Export/ac_detrend_ret.csv']

In [339]:
sharpe_files= loop_files('sharpes')#'sharpes_monthly_diff_OCO')

In [340]:
iter_DB = list(
    itertools.product(
        scaler, #0
        PCA_components, #1
        min_samples, #2
        metric, #3
        eps, #4
        algorithm, #5
        n_jobs, #6
        ratios_files,  #7
        sharpe_files,#8
        p)) #9

In [341]:
len(iter_DB)

7776

### DBSCAN Loop for all Param Combos

In [342]:
sil_master_DB = []
predicted_labels_master_DB = []
portfolio_companies_master_DB = []
excess_master_DB = []
combo_excess_masterlist_DB = []

for combo in iter_DB:
    movements = read_returns(combo[7])

    nrow = movements.shape[0]
    ncol = movements.shape[1]

    predicted_labels = DBSCAN_DS(movements,
                                 eps=combo[4],
                                 min_samples=combo[2],
                                 metric=combo[3],
                                 algorithm=combo[5],
                                 n_jobs=combo[6],
                                 scaler=combo[0],
                                 PCA_components=combo[1],
                                 p=combo[9])

    portfolio_companies, excess = SUMMARYSTATS_DS(predicted_labels, combo[7],
                                                  combo[8])

    #sil_master_DB.append(sil_score)
    predicted_labels_master_DB.append(predicted_labels)
    portfolio_companies_master_DB.append(portfolio_companies)
    excess_master_DB.append(excess)
    combo_excess_masterlist_DB.append([combo, excess])

### DBSCAN Results

In [343]:
max(excess_master_DB)

0.1051473567901044

In [344]:
max_index = excess_master_DB.index(max(excess_master_DB))

In [345]:
portfolio_companies_master_DB[max_index]

Index(['PGR', 'XEL'], dtype='object')

In [346]:
iter_DB[max_index]

(MinMaxScaler(),
 8,
 10,
 'minkowski',
 0.256098350775273,
 'auto',
 -1,
 'Export/ratios.csv',
 'Export/sharpes_diff_OCO.csv',
 1.5)

## OPTICS

### OPTICS Params

In [379]:
#OPTICS Params
scaler = [MinMaxScaler(),MaxAbsScaler()]
PCA_components = [2,3,5,4,11]
random_state= [10]
min_samples= [1,2,3,4,16]
max_eps = inf
metric= ['minkowski']
p=2,
metric_params= None 
cluster_method= 'xi' 
eps= [0.0633380090583151,
0.42223]
xi= [0.05]
predecessor_correction = [True]
min_cluster_size = [0.05]
algorithm = ['auto'] 
leaf_size= [30] 


In [380]:
ratios_files = loop_files('ratios')

In [381]:
sharpe_files= ['Export/sharpes_monthly_diff_OCO.csv']#loop_files('sharpe')

In [382]:
iter_OPT = list(
    itertools.product(
        scaler, #0
        PCA_components, #1
        min_samples, #2
        metric, #3
        eps, #4
        xi,#5
        min_cluster_size,#6
        algorithm, #7
        ratios_files,  #8
        sharpe_files,#9
        p)) #10

In [383]:
len(iter_OPT)

100

### OPTICS Loop for all Combos of Params

In [373]:
sil_master_OPT = []
predicted_labels_master_OPT = []
portfolio_companies_master_OPT = []
excess_master_OPT = []
combo_excess_masterlist_OPT = []

for combo in iter_OPT:
    movements = read_returns(combo[8])

    nrow = movements.shape[0]
    ncol = movements.shape[1]

    predicted_labels = OPTICS_DS(movements,
                                            scaler=combo[0],
                                            PCA_components=combo[1],
                                            min_samples=combo[2],
                                            metric=combo[3],
                                            eps=combo[4],
                                            xi=combo[5],
                                            min_cluster_size=combo[6])
    
    predicted_labels = predicted_labels[predicted_labels['group'] > -1]
    #  ^gets rid of outliers group
    portfolio_companies, excess = SUMMARYSTATS_DS(predicted_labels, combo[8],
                                                  combo[9])

    #sil_master_DB.append(sil_score)
    predicted_labels_master_OPT.append(predicted_labels)
    portfolio_companies_master_OPT.append(portfolio_companies)
    excess_master_OPT.append(excess)
    combo_excess_masterlist_OPT.append([combo,excess])

### OPTICS Results

In [374]:
max(excess_master_OPT)

0.08770828264743336

In [375]:
max_index = excess_master_OPT.index(max(excess_master_OPT))

In [376]:
max_index

460

In [377]:
portfolio_companies_master_OPT[max_index]

Index(['XEL'], dtype='object')

In [378]:
iter_OPT[max_index]

(MinMaxScaler(),
 21,
 10,
 'manhattan',
 0.0633380090583151,
 0.05,
 0.05,
 'auto',
 'Export/ratios.csv',
 'Export/sharpes_monthly_diff_OCO.csv',
 2)